In [1]:
import pandas as pd
import json
import requests
import datetime
import time
import math
from dateutil.relativedelta import relativedelta
import base64
from typing import Optional, Tuple

In [2]:
import os
from dotenv import load_dotenv

load_dotenv()
kindred_bearer_token = os.environ.get('KINDRED_BEARER_TOKEN')
openrouteservice_api_key = os.environ.get('OPEN_ROUTE_SERVICE_KEY')
my_email = os.environ.get('EMAIL')

In [17]:
class _TokenBox:
    """Holds the current access token so it can be updated when refreshed."""
    def __init__(self, access_token: str):
        self.access = access_token

token_box = _TokenBox(kindred_bearer_token)

def _build_headers() -> dict:
    return {
        "accept": "*/*",
        "content-type": "application/json",
        "apollographql-client-name": "Web",
        "apollographql-client-version": "1.929.3",
        "authorization": f"Bearer {token_box.access}",
        "origin": "https://livekindred.com",
        "referer": "https://livekindred.com/",
        "x-locale": "en",
    }

# otp flow
MUT_SEND_EMAIL = """
mutation sendMagicLinkOrOTP($email: String!, $path: String) {
  startEmailLoginUser(email: $email, path: $path) {
    mode
    length
  }
}
"""

MUT_FINISH_EMAIL = """
mutation FinishEmailLoginUser($deviceId: String, $email: String!, $emailToken: String!) {
  finishEmailLoginUser(deviceId: $deviceId, email: $email, emailToken: $emailToken) {
    accessToken
    refreshToken
  }
}
"""

def gql_noauth(operation_name: str, query: str, variables: dict):
    """Send a GraphQL request without Authorization (for login)."""
    headers = {
        "content-type": "application/json",
        "origin": "https://livekindred.com",
        "referer": "https://livekindred.com/",
        "apollographql-client-name": "Web",
        "apollographql-client-version": "1.929.3",
        "x-locale": "en",
    }
    payload = {"operationName": operation_name, "query": query, "variables": variables}
    r = requests.post(kindred_url, headers=headers, data=json.dumps(payload))
    r.raise_for_status()
    j = r.json()
    if "errors" in j:
        raise RuntimeError(json.dumps(j["errors"], indent=2))
    return j["data"]


def send_email_otp(email: str, path: str = "/explore") -> dict:
    """Triggers the OTP email."""
    data = gql_noauth("sendMagicLinkOrOTP", MUT_SEND_EMAIL, {"email": email, "path": path})
    return data["startEmailLoginUser"]


def finish_email_login(email: str, email_token: str, device_id=None):
    """Completes login with the OTP code and returns new tokens."""
    data = gql_noauth(
        "FinishEmailLoginUser",
        MUT_FINISH_EMAIL,
        {"deviceId": device_id, "email": email, "emailToken": email_token},
    )
    return data["finishEmailLoginUser"]["accessToken"], data["finishEmailLoginUser"]["refreshToken"]


def refresh_fn():
    """
    Re-run the email OTP login flow to get a new access token.
    This function will be called automatically by post_graphql() when auth fails.
    """
    email = my_email
    path = "/explore"
    print(f"\nAccess token expired — sending new OTP to {email}...")
    meta = send_email_otp(email, path)
    print(f"Mode={meta['mode']}, expected length={meta['length']}")
    otp = input("Enter OTP from email: ").strip()
    new_access, new_refresh = finish_email_login(email, otp)
    print("New access token obtained.")
    return new_access

# Ikon info

In [18]:
# csv created by chatgpt agent using epicorikon.com
locations = pd.read_csv('resort_locations_20256.csv')
locations.columns = locations.columns.str.lower()
locations.rename(columns={'resortregion': 'region'
    , 'stateorprovince': 'state'
    , 'skiableacres': 'skiable_acres'
    , 'verticaldrop': 'vertical_drop'
    , 'annualsnowfall': 'annual_snowfall'}, inplace=True)

# Drive time

In [19]:
headers = {
    'Accept': 'application/json, application/geo+json, application/gpx+xml, img/png; charset=utf-8',
    'api_key': openrouteservice_api_key,
    'Content-Type': 'application/json; charset=utf-8'
}

In [20]:
def get_driving_time(resort_lat, resort_lon, house_lat, house_lon):
    
    coordinates = {"coordinates":[[house_lon,house_lat],[resort_lon,resort_lat]]}
    directions_url = 'https://api.openrouteservice.org/v2/directions/driving-car'
    directions_url_and_key = directions_url + '?api_key=' + openrouteservice_api_key
    call = requests.post(directions_url_and_key, json=coordinates, headers=headers)
    
    return call.json()['routes'][0]['summary']['duration'] / 60

def safe_get_driving_time(resort_lat, resort_lon, house_lat, house_lon):
        try:
            return get_driving_time(resort_lat, resort_lon, house_lat, house_lon)
        except KeyError as e:
            if str(e) == "'routes'":
                return None
            else:
                raise
        except Exception:
            return None

# Kindred info

In [21]:
kindred_url = "https://app.livekindred.com/api/graphql"

In [22]:
def post_graphql(operation_name: str, query: str, variables: dict, refresh_fn=refresh_fn):
    """
    Posts a GraphQL query/mutation. If auth fails (401/403 or GraphQL UNAUTHENTICATED),
    refreshes the access token using refresh_fn() and retries once.
    """
    payload = {"operationName": operation_name, "query": query, "variables": variables or {}}

    def _do():
        return requests.post(kindred_url, headers=_build_headers(), data=json.dumps(payload))

    r = _do()

    # HTTP auth errors → refresh
    if r.status_code in (401, 403):
        token_box.access = refresh_fn()
        r = _do()

    def parse(resp):
        try:
            d = resp.json()
        except ValueError:
            return None, None
        return d, d.get("errors")

    data, errs = parse(r)

    # If HTTP error, check GraphQL errors first
    if r.status_code >= 400:
        if errs and any((e.get("extensions") or {}).get("code") == "UNAUTHENTICATED" for e in errs):
            token_box.access = refresh_fn()
            r = _do()
            data, errs = parse(r)
            if r.status_code >= 400:
                r.raise_for_status()
            if errs:
                raise RuntimeError(json.dumps(errs, indent=2))
            return data["data"]
        if errs:
            raise RuntimeError(json.dumps(errs, indent=2))
        r.raise_for_status()

    # Success status but GraphQL UNAUTHENTICATED
    if errs:
        if any((e.get("extensions") or {}).get("code") == "UNAUTHENTICATED" for e in errs):
            token_box.access = refresh_fn()
            r = _do()
            data, errs = parse(r)
            if errs:
                raise RuntimeError(json.dumps(errs, indent=2))
            return data["data"]
        raise RuntimeError(json.dumps(errs, indent=2))

    return data["data"]

In [23]:
query_explore_list = """
query exploreList($filter: FlexibleSearchFilter!, $pagination: Pagination!, $sortedAt: Date!, $width: Int!, $avatarWidth: Int!) {
  getHomesWithSearchCriteria(filter: $filter, pagination: $pagination, sortedAt: $sortedAt) {
    page
    hasMore
    didFindPerfectMatches
    homeRecs {
      home {
        ...HomeCardData
      }
      matchingStatus { ...SearchResultScore }
      household { ...SearchHousehold }
    }
  }
}

fragment HomeCardData on Home {
  id
  status
  destination { id name region }
  media { url thumbnailUrl(width: $width) }
  title
  titleV2 { translation originalLanguage }
  availabilitiesWithoutBookedDates { ...HomeAvailability }
  swapAvailabilitiesV2 { ...SwapAvailabilities }
  isFavorite
  homeProfileProgress
  maxGuestsLimit
  workspacesCount
  bathrooms
  bedroomsCount
  bedsCount
  petPreference
  petHostingDetails
  lat
  lon
  preSelect { ...PreSelectDates }
  swapQuality
  owner { id displayName isOpenForInquiry }
  swapAvailabilitiesV2 { destinationIds destinationNames }
  excludeHomeMediaFromMarketing
  pricingPreview { ...PublicPreviewPricing }
  restrictionReasons
}

fragment HomeAvailability on HomeAvailability { id homeId startDate endDate }
fragment SwapAvailabilities on HomeSwapAvailability {
  id swapAvailabilityId start end tripLengthsV2 minimumNights dateRanges
  destinationIds destinationNames destinationName travelPlanId
  travelPlan {
    tripTypes { ...HomeTripType }
    minBedrooms minBathrooms minBeds totalGuests
    homeFilters { ...HomeFilters }
  }
  swapDestination { name }
}
fragment HomeTripType on HomeTripType { name displayName photoUrl }
fragment HomeFilters on HomeFilter {
  ... on AmenityFilter { __typename amenity enabled }
  ... on PetPreferenceFilter { __typename petPreference enabled }
  ... on BedTypeFilter { __typename bed enabled }
  ... on CompositeHomeFilter { __typename compositeFilter enabled }
}
fragment PreSelectDates on PreSelect { dateRange isSwap }
fragment PublicPreviewPricing on PricingPreviewPublic {
  fees { ...TripPricingFee }
  totalMoney { ...TripMoneyDisplay }
  moneyPerNight { ...TripMoneyDisplay }
  credit { totalCredits }
  pricingComparison { totalMoney { amount currency } moneyPerNight { ...TripMoneyDisplay } totalNights }
}
fragment TripPricingFee on PricingFee { type total { amount currency } }
fragment TripMoneyDisplay on MoneyDisplay { amount currency displayString }
fragment SearchResultScore on MatchingResult {
  alternateDates alternateLocation lessBedrooms lessWorkstations lessBeds lessHomeCapacity noPetsAllowed needPetsFriendlyHomeOtherSide score
}
fragment SearchHousehold on BaseHouseholdProfile {
  id
  primaryResident { displayName image { url(width: $avatarWidth) } }
  householdImages { url(width: $avatarWidth) }
}
"""

In [24]:
def make_polygon(lat, lon, distance_miles):
    """
    Creates an octagonal polygon around (lat, lon) with approximately
    'distance_miles' as the distance from center to each vertex,
    correcting for Earth's curvature so the octagon is roughly equilateral
    (i.e., not stretched at higher latitudes).
    """

    lat_rad = math.radians(lat)
    miles_per_lat = 69.0 # one degree latitude is ~69 miles (everywhere)
    miles_per_lon = 69.172 * math.cos(lat_rad) # one degree longitude is 69.172 * cos(latitude)

    # angles for the 8 directions
    directions_deg = [90, 45, 0, 315, 270, 225, 180, 135]
    polygon = []
    for angle in directions_deg:
        angle_rad = math.radians(angle)
        d_lat = (distance_miles * math.sin(angle_rad)) / miles_per_lat
        d_lon = (distance_miles * math.cos(angle_rad)) / miles_per_lon
        point = {
            "lat": lat + d_lat,
            "lon": lon + d_lon,
        }
        polygon.append(point)
    return polygon

def to_iso_date_range(start_date, end_date):
    """
    Accepts string or datetime for start_date and end_date.
    Returns [start_iso, end_iso] with format: "%Y-%m-%dT%H:%M:%S.000Z".
    """
    from datetime import datetime

    iso_format = "%Y-%m-%dT%H:%M:%S.000Z"

    def ensure_dt(d):
        if isinstance(d, str):
            return datetime.strptime(d, "%Y-%m-%d")
        return d

    dt_start = ensure_dt(start_date)
    dt_end = ensure_dt(end_date)

    return [dt_start.strftime(iso_format), dt_end.strftime(iso_format)]

def make_monthly_date_ranges(start_date, end_date, date_format="%Y-%m-%d"):
    """
    Returns a list of one-month date ranges between start_date and end_date, in ISO format.
    """
    from dateutil.relativedelta import relativedelta
    from datetime import datetime

    current_date = datetime.strptime(start_date, date_format)
    end_date_dt = datetime.strptime(end_date, date_format)

    date_ranges = []
    while current_date < end_date_dt:
        next_month = current_date + relativedelta(months=1)
        range_end = min(next_month, end_date_dt)
        date_ranges.append(to_iso_date_range(current_date, range_end))
        current_date = range_end
    return date_ranges


In [31]:
def explore_map_once(polygon, date_range, date_type='flexible', min_nights=0, page=0, page_size=50):
    if date_type == 'exact':
        trip_type = "EXACT_DATES"
    else:
        trip_type = "MINIMUM_NIGHTS"

    variables = {
        "filter": {
            "tripLengthsV2": [trip_type],
            **({"minimumNights": min_nights} if date_type == 'flexible' else {}),
            "dateRanges": date_range,
            "includeCloseDates": False,
            "isFavoriteHomesOnly": False,
            "onboardingSort": False,
            "polygon": polygon,
            "matchTypes": ["SWAP", "AVAILABILITY"],
            "minBedrooms": 0,
            "minBathrooms": 0,
            "minBeds": 0,
            "totalGuests": 0,
            "filterInput": {
                "amenityFilters": [],
                "bedTypeFilters": [],
                "compositeFilters": [],
                "petPreferences": []
            }
        },
        "pagination": {"page": page, "pageSize": page_size},
        "sortedAt": datetime.datetime.now(datetime.UTC).isoformat().replace("+00:00", "Z"),
        "width": 720,
        "avatarWidth": 90
    }
    return variables

In [32]:
def explore_map_multiple(
    lat,
    lon,
    distance_miles,
    date_range,
    date_type,
    min_nights=0,
    page_size=50
):
    
    sorted_at = datetime.datetime.now(datetime.UTC).isoformat(timespec='milliseconds').replace("+00:00", "Z")
    all_rows = []
    page = 0
    
    polygon = make_polygon(lat, lon, distance_miles)
    
    if date_type == 'flexible':
        formatted_date_range = make_monthly_date_ranges(date_range[0], date_range[1])
    else:
        formatted_date_range = [to_iso_date_range(date_range[0], date_range[1])]

    while True:
        vars_page = explore_map_once(
            polygon,
            formatted_date_range,
            date_type=date_type,
            min_nights=min_nights,
            page=page,
            page_size=page_size
        )

        vars_page["sortedAt"] = sorted_at  # keep stable per crawl

        data = post_graphql("exploreList", query_explore_list, vars_page)

        res = data["getHomesWithSearchCriteria"]

        for rec in res.get("homeRecs", []):
            h = rec["home"]
            all_rows.append({
                "id": h.get("id"),
                "homeId": h.get("id"),
                "title": h.get("title"),
                "destination": (h.get("destination") or {}).get("name"),
                "lat": h.get("lat"),
                "lon": h.get("lon"),
                "availabilitiesWithoutBookedDates": h.get("availabilitiesWithoutBookedDates", []),
                "maxGuestsLimit": h.get("maxGuestsLimit"),
                "petPreference": h.get("petPreference"),
            })
    
        if not res.get("hasMore"):
            break
        
        page += 1
        time.sleep(0.4)  # be polite

    return all_rows

def format_results(results):
    results_df = pd.DataFrame(results)
    
    # add error handling for missing 'availabilitiesWithoutBookedDates'
    if 'availabilitiesWithoutBookedDates' not in results_df.columns:
        results_df['availabilitiesWithoutBookedDates'] = [[] for _ in range(len(results_df))]
    
    results_df['availabilitiesWithoutBookedDates'] = results_df['availabilitiesWithoutBookedDates'].apply(
        lambda avails: [{'start': av['startDate'], 'end': av['endDate']} for av in avails] if avails else []
    )

    # clickable url based on homeId
    if 'homeId' in results_df.columns:
        results_df['home_url'] = results_df['homeId'].apply(lambda id: f'https://livekindred.com/home/{id}')
    else:
        results_df['home_url'] = None

    results_df.rename(
        columns={
            'homeId': 'home_id',
            'maxGuestsLimit': 'guests',
            'petPreference': 'pet_preference',
            'availabilitiesWithoutBookedDates': 'availabilities',
            'lat': 'latitude',
            'lon': 'longitude'
        },
        inplace=True
    )

    return results_df


# Put it all together

In [27]:
def resort_map_multiple(
    resort_df,
    date_range,
    date_type='flexible',
    mile_range=35,
    page_size=50,
    min_nights=0
):
    """
    Given a dataframe of resorts and a date range, returns a dataframe of Kindredhomes near the resorts.
    Add a column for the driving time to the resorts.

    Parameters:
        resort_df (pd.DataFrame): DataFrame of resorts with 'latitude', 'longitude', 'resort', 'state' columns.
        date_range (list or tuple): [start_date, end_date] as strings, e.g. ['2026-01-17', '2026-01-19'].
        mile_range (int, optional): Search "radius" in miles. Not a true radius since the search range is a square (corners will be farther than mile_range).Default is 35.
        page_size (int, optional): Number of results per API page. Default is 50.
        min_nights (int, optional): Minimum number of nights for stay. Default is 0.
    """
    
    results_full = pd.DataFrame()

    for idx, row in resort_df.iterrows():
        current_time = datetime.datetime.now()
        
        results = explore_map_multiple(
            lat=row['latitude'],
            lon=row['longitude'],
            distance_miles=mile_range,
            date_range=date_range,
            date_type=date_type,
            min_nights=min_nights,
            page_size=page_size
        )
        
        results_df = format_results(results)
        results_df['resort'] = row['resort']
        results_df['state'] = row['state']
        results_df['region'] = row['region']
        results_df['resort_lat'] = row['latitude']
        results_df['resort_lon'] = row['longitude']
        
        results_full = pd.concat([results_full, results_df])
    
        time_elapsed = datetime.datetime.now() - current_time
        print(f"Found {len(results_df)} houses near {row.get('resort', idx)} in {time_elapsed}")

    results_full['driving_time_minutes'] = results_full.apply(
        lambda row: safe_get_driving_time(row['resort_lat'], row['resort_lon'], row['latitude'], row['longitude']),
        axis=1
    )
    
    if results_full.empty:
        print("No homes found for any resort. Try adjusting your date range, mile range, or check your data.")

    return results_full.sort_values(by='driving_time_minutes')

# Using it

In [28]:
northeast_locations = locations[locations['region'] == 'Northeast']

In [ ]:
northeast_results = resort_map_multiple(northeast_locations, ['2026-01-17', '2026-01-19'], mile_range=60)

Found 1 houses near Cranmore in 0:00:00.405283
Found 0 houses near Jiminy Peak in 0:00:00.195873
Found 3 houses near Killington in 0:00:00.330239
Found 0 houses near Le Massif de Charlevoix in 0:00:00.365766
Found 2 houses near Loon Mountain in 0:00:00.329328
Found 3 houses near Pico in 0:00:00.373094
Found 2 houses near Stratton in 0:00:00.247318
Found 5 houses near Sugarbush in 0:00:00.322137
Found 0 houses near Sugarloaf in 0:00:00.277430
Found 0 houses near Sunday River in 0:00:00.201427
Found 0 houses near Tremblant in 0:00:00.206885


In [ ]:
with pd.option_context('display.max_colwidth', None):
    display(northeast_results.sort_values(by='driving_time_minutes').head(10))

,id,home_id,title,destination,latitude,longitude,availabilities,guests,pet_preference,home_url,resort,state,region,resort_lat,resort_lon,driving_time_minutes
0,clqeh23du00pykv3ckajh3kgy,clqeh23du00pykv3ckajh3kgy,Peaceful & Sunny Lakeside Condo,Meredith,43.656831,-71.489985,"[{'start': '2025-10-20', 'end': '2026-04-30'}]",4.0,NO,https://livekindred.com/home/clqeh23du00pykv3ckajh3kgy,Loon Mountain,New Hampshire,Northeast,44.05644,-71.63545,56.203333
3,cm6r9bhw62mby2aouhdo3c582,cm6r9bhw62mby2aouhdo3c582,Welcome to our lakeside retreat with sauna.,Hinesburg,44.352431,-73.084339,"[{'start': '2026-01-06', 'end': '2026-03-31'}]",6.0,MAYBE,https://livekindred.com/home/cm6r9bhw62mby2aouhdo3c582,Sugarbush,Vermont,Northeast,44.13526,-72.89562,61.728333
1,cmalrshzs0cog8s4reg2e4160,cmalrshzs0cog8s4reg2e4160,Rustic Vermont Farm Stay - Birch House,Wells,43.429798,-73.130898,"[{'start': '2026-01-06', 'end': '2026-04-30'}]",2.0,NO,https://livekindred.com/home/cmalrshzs0cog8s4reg2e4160,Pico,Vermont,Northeast,43.66180,-72.84333,65.005000
0,cmg9ymoos007jm22aom4je9d0,cmg9ymoos007jm22aom4je9d0,Rustic Vermont Farm Stay - Shady Grove,Wells,43.433398,-73.131573,"[{'start': '2026-01-06', 'end': '2026-04-30'}]",2.0,NO,https://livekindred.com/home/cmg9ymoos007jm22aom4je9d0,Pico,Vermont,Northeast,43.66180,-72.84333,66.096667
1,cmalrshzs0cog8s4reg2e4160,cmalrshzs0cog8s4reg2e4160,Rustic Vermont Farm Stay - Birch House,Wells,43.429798,-73.130898,"[{'start': '2026-01-06', 'end': '2026-04-30'}]",2.0,NO,https://livekindred.com/home/cmalrshzs0cog8s4reg2e4160,Killington,Vermont,Northeast,43.62573,-72.79827,73.528333
0,cmg9ymoos007jm22aom4je9d0,cmg9ymoos007jm22aom4je9d0,Rustic Vermont Farm Stay - Shady Grove,Wells,43.433398,-73.131573,"[{'start': '2026-01-06', 'end': '2026-04-30'}]",2.0,NO,https://livekindred.com/home/cmg9ymoos007jm22aom4je9d0,Killington,Vermont,Northeast,43.62573,-72.79827,74.620000
0,clqeh23du00pykv3ckajh3kgy,clqeh23du00pykv3ckajh3kgy,Peaceful & Sunny Lakeside Condo,Meredith,43.656831,-71.489985,"[{'start': '2025-10-20', 'end': '2026-04-30'}]",4.0,NO,https://livekindred.com/home/clqeh23du00pykv3ckajh3kgy,Cranmore,New Hampshire,Northeast,44.05638,-71.11078,78.461667
1,cmalrshzs0cog8s4reg2e4160,cmalrshzs0cog8s4reg2e4160,Rustic Vermont Farm Stay - Birch House,Wells,43.429798,-73.130898,"[{'start': '2026-01-06', 'end': '2026-04-30'}]",2.0,NO,https://livekindred.com/home/cmalrshzs0cog8s4reg2e4160,Stratton,Vermont,Northeast,43.11342,-72.91032,79.236667
0,cmg9ymoos007jm22aom4je9d0,cmg9ymoos007jm22aom4je9d0,Rustic Vermont Farm Stay - Shady Grove,Wells,43.433398,-73.131573,"[{'start': '2026-01-06', 'end': '2026-04-30'}]",2.0,NO,https://livekindred.com/home/cmg9ymoos007jm22aom4je9d0,Stratton,Vermont,Northeast,43.11342,-72.91032,80.328333
2,cm6r9bhw62mby2aouhdo3c582,cm6r9bhw62mby2aouhdo3c582,Welcome to our lakeside retreat with sauna.,Hinesburg,44.352431,-73.084339,"[{'start': '2026-01-06', 'end': '2026-03-31'}]",6.0,MAYBE,https://livekindred.com/home/cm6r9bhw62mby2aouhdo3c582,Pico,Vermont,Northeast,43.66180,-72.84333,108.841667


In [20]:
europe_locations = locations[locations['region'] == 'Europe']

In [49]:
europe_results = resort_map_multiple(
    europe_locations,
    ['2025-12-27', '2026-01-04'],
    date_type='exact',
    mile_range=40
)

Found 0 houses near Cervino Ski Paradise in 0:00:00.384896
Found 0 houses near Chamonix Mont-Blanc in 0:00:00.271733
Found 0 houses near Courmayeur Mont Blanc in 0:00:00.460980
Found 1 houses near Dolomiti Superski in 0:00:00.376140
Found 0 houses near Grandvalira in 0:00:00.179913
Found 0 houses near Ischgl in 0:00:00.176776
Found 1 houses near Kitzbühel in 0:00:00.280840
Found 1 houses near La Thuile - Espace San Bernardo in 0:00:00.262700
Found 2 houses near Monterosa Ski in 0:00:00.356572
Found 1 houses near Pila in 0:00:00.337836
Found 0 houses near St. Moritz in 0:00:00.320675
Found 0 houses near Zermatt in 0:00:00.241877


In [50]:
with pd.option_context('display.max_colwidth', None):
    display(europe_results.sort_values(by='driving_time_minutes').head(10))

,availabilities,home_url,resort,state,region,resort_lat,resort_lon,id,home_id,title,destination,latitude,longitude,guests,pet_preference,driving_time_minutes
1,"[{'start': '2025-12-25', 'end': '2026-01-08'}]",https://livekindred.com/home/cmckzja520yk213lc9na2ma6v,Monterosa Ski,NaN,Europe,45.85469,7.73321,cmckzja520yk213lc9na2ma6v,cmckzja520yk213lc9na2ma6v,Torino home,Torino,45.059166,7.684403,2.0,MAYBE,96.016667
0,"[{'start': '2025-12-20', 'end': '2026-01-04'}]",https://livekindred.com/home/cmciuk4ql3ern365dto90dzoo,Kitzbühel,NaN,Europe,47.44351,12.38964,cmciuk4ql3ern365dto90dzoo,cmciuk4ql3ern365dto90dzoo,Sunny Stream - your Salzburg home,Salzburg,47.815949,13.022196,3.0,NO,101.408333
0,"[{'start': '2025-12-25', 'end': '2026-01-08'}]",https://livekindred.com/home/cmckzja520yk213lc9na2ma6v,Pila,NaN,Europe,45.69021,7.31018,cmckzja520yk213lc9na2ma6v,cmckzja520yk213lc9na2ma6v,Torino home,Torino,45.059166,7.684403,2.0,MAYBE,112.845000
0,"[{'start': '2025-12-25', 'end': '2026-01-08'}]",https://livekindred.com/home/cmckzja520yk213lc9na2ma6v,La Thuile - Espace San Bernardo,NaN,Europe,45.71242,6.94892,cmckzja520yk213lc9na2ma6v,cmckzja520yk213lc9na2ma6v,Torino home,Torino,45.059166,7.684403,2.0,MAYBE,128.555000
0,"[{'start': '2025-10-26', 'end': '2026-01-31'}]",https://livekindred.com/home/cm6f39nyo1a9achkolohf3w2l,Monterosa Ski,NaN,Europe,45.85469,7.73321,cm6f39nyo1a9achkolohf3w2l,cm6f39nyo1a9achkolohf3w2l,"Spacious home near Como, Milan and Switzerland.",Malnate,45.787671,8.868370,4.0,MAYBE,138.576667
0,"[{'start': '2025-12-25', 'end': '2026-01-04'}]",https://livekindred.com/home/clztlmmam254njo1x2gonveja,Dolomiti Superski,NaN,Europe,46.53333,12.13333,clztlmmam254njo1x2gonveja,clztlmmam254njo1x2gonveja,Peaceful family-friendly home with big garden,Valbrenta,45.839544,11.689484,8.0,MAYBE,142.795000


In [14]:
medium_plus_locations = locations[
    (locations['skiable_acres'] >= 350) &
    (locations['annual_snowfall'] >= 150)
]

In [ ]:
medium_plus_results = resort_map_multiple(
    medium_plus_locations,
    ['2026-01-17', '2026-01-19'],
    date_type='exact',
    mile_range=50
)


In [17]:
with pd.option_context('display.max_colwidth', None):
    display(medium_plus_results.sort_values(by='driving_time_minutes').head(40))

,id,home_id,title,destination,latitude,longitude,availabilities,guests,pet_preference,home_url,resort,state,resort_lat,resort_lon,driving_time_minutes
2,cm7pnegd92dgaua71wkws2jku,cm7pnegd92dgaua71wkws2jku,Cozy Sandy home close to mountains,Sandy,40.545481,-111.853475,"[{'start': '2026-01-15', 'end': '2026-01-19'}]",4.0,NO,https://livekindred.com/home/cm7pnegd92dgaua71wkws2jku,Alta,Utah,40.58827,-111.63799,28.720000
11,clxqab08t339t2sbcsf7y7bov,clxqab08t339t2sbcsf7y7bov,"Peaceful, Open and Sunny Downtown Vancouver Nest!",Vancouver,49.283194,-123.136111,"[{'start': '2026-01-15', 'end': '2026-01-26'}]",2.0,NO,https://livekindred.com/home/clxqab08t339t2sbcsf7y7bov,Cypress Mountain,British Columbia,49.39602,-123.20681,37.146667
3,clxvvq4mc0uaixpkpximc84j4,clxvvq4mc0uaixpkpximc84j4,Central and spacious Yaletown Vancouver condo,Vancouver,49.279504,-123.118394,"[{'start': '2025-12-11', 'end': '2026-03-31'}]",3.0,NO,https://livekindred.com/home/clxvvq4mc0uaixpkpximc84j4,Cypress Mountain,British Columbia,49.39602,-123.20681,37.721667
3,clixdrhjs06usm33a0vhettz1,clixdrhjs06usm33a0vhettz1,Peaceful and sunny Sugar House home,Salt Lake City,40.727282,-111.851052,"[{'start': '2025-12-01', 'end': '2026-01-29'}]",2.0,MAYBE,https://livekindred.com/home/clixdrhjs06usm33a0vhettz1,Deer Valley,Utah,40.63738,-111.48049,38.791667
0,cm560fcoe0jjwjalp4jxwc03y,cm560fcoe0jjwjalp4jxwc03y,Peaceful and sunny Central City home,Salt Lake City,40.704993,-111.875864,"[{'start': '2026-01-08', 'end': '2026-02-20'}]",4.0,MAYBE,https://livekindred.com/home/cm560fcoe0jjwjalp4jxwc03y,Deer Valley,Utah,40.63738,-111.48049,39.891667
3,clixdrhjs06usm33a0vhettz1,clixdrhjs06usm33a0vhettz1,Peaceful and sunny Sugar House home,Salt Lake City,40.727282,-111.851052,"[{'start': '2025-12-01', 'end': '2026-01-29'}]",2.0,MAYBE,https://livekindred.com/home/clixdrhjs06usm33a0vhettz1,Alta,Utah,40.58827,-111.63799,40.003333
0,cm560fcoe0jjwjalp4jxwc03y,cm560fcoe0jjwjalp4jxwc03y,Peaceful and sunny Central City home,Salt Lake City,40.704993,-111.875864,"[{'start': '2026-01-08', 'end': '2026-02-20'}]",4.0,MAYBE,https://livekindred.com/home/cm560fcoe0jjwjalp4jxwc03y,Alta,Utah,40.58827,-111.63799,41.105000
0,cm7o64lfo4ix6lv5gbgzoza7j,cm7o64lfo4ix6lv5gbgzoza7j,Rabbit Creek home,Anchorage,61.069359,-149.792584,"[{'start': '2025-10-16', 'end': '2026-03-01'}]",8.0,MAYBE,https://livekindred.com/home/cm7o64lfo4ix6lv5gbgzoza7j,Alyeska,Alaska,60.97050,-149.09820,42.690000
13,cmfcy7c4801nekk2jncm9sq9w,cmfcy7c4801nekk2jncm9sq9w,Peaceful and spacious 1-bedroom home 750 Sq.Ft,Vancouver,49.258707,-123.143547,"[{'start': '2025-10-16', 'end': '2026-04-30'}]",2.0,MAYBE,https://livekindred.com/home/cmfcy7c4801nekk2jncm9sq9w,Cypress Mountain,British Columbia,49.39602,-123.20681,43.910000
1,cm96dpm9993u4lio8oe42u03j,cm96dpm9993u4lio8oe42u03j,Spacious and Historic Downtown Loft,Salt Lake City,40.767432,-111.899361,"[{'start': '2026-01-17', 'end': '2026-01-24'}]",4.0,MAYBE,https://livekindred.com/home/cm96dpm9993u4lio8oe42u03j,Deer Valley,Utah,40.63738,-111.48049,45.768333


# Resort stats

In [5]:
locations.head()

,resort,region,state,latitude,longitude,skiable_acres,vertical_drop,annual_snowfall
0,Alta,Rockies,Utah,40.58827,-111.63799,2614,2538,545
1,Alyeska,West Coast,Alaska,60.97050,-149.09820,1400,3200,409
2,Arapahoe Basin,Rockies,Colorado,39.64167,-105.87167,1428,2530,350
3,Aspen Highlands,Rockies,Colorado,39.18226,-106.85768,1010,3638,300
4,Aspen Mountain,Rockies,Colorado,39.18626,-106.82042,673,3267,300


In [10]:
import plotly.express as px

sorted_locations = locations.sort_values('skiable_acres', ascending=False)
fig = px.scatter(
    sorted_locations,
    x='skiable_acres',
    y='vertical_drop',
    color='region',
    title='Resorts by Skiable Acres and Vertical Drop',
    category_orders={'resort': sorted_locations['resort'].tolist()},
    hover_data=['resort', 'vertical_drop', 'skiable_acres']
)
fig.show()


In [11]:
sorted_locations = locations.sort_values('skiable_acres', ascending=False)
fig = px.scatter(
    sorted_locations,
    x='skiable_acres',
    y='annual_snowfall',
    color='region',
    title='Resorts by Skiable Acres and Annual Snowfall',
    category_orders={'resort': sorted_locations['resort'].tolist()},
    hover_data=['resort', 'annual_snowfall', 'skiable_acres']
)
fig.show()

# To do